In [2]:
# import prompt
from constants import DATABASE_PATH_PATTERN, DEV_JSON_PATH
import json

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
with open(DEV_JSON_PATH, "r") as input_file:
    dataset = json.load(input_file)
with open("./0319_step_2.json","r") as input_file:
    pred_dataset = json.load(input_file)
print(len(pred_dataset))
assert(len(dataset) == len(pred_dataset))

1034


In [4]:
pred_output = []
for pred in pred_dataset[:]:
    pred_output.append(pred[pred.find("SELECT "):    ])
    # lines = pred.split("\n")[1:]
    # table ={}
    # for line in lines:
    #     pos = line.find(":")
    #     table_name = line[:pos]
    #     columns = line[pos + 3 : -1].strip().split(",")
    #     if columns[0] == "":
    #         columns=list()
    #     table[table_name] = set(columns)
    # pred_output.append(table)
with open("./0314_step_2.json", "w") as output_file:
    json.dump(pred_output, output_file,)

In [7]:
from sql_metadata import Parser
import sqlparse


def equal(pred: dict[str, set], gold: dict[str, set]) -> bool:
    return pred == gold


def table_contain(pred: dict[str, set], gold: dict[str, set]) -> bool:
    return set(gold.keys()).issubset(set(pred.keys()))


def contain(pred: dict[str, set], gold: dict[str, set]) -> bool:
    for table, columns in gold.items():
        if table in pred:
            if columns.issubset(pred[table]):
                continue
            else:
                return False
        else:
            return False
    return True


equal_cnt = 0
contain_cnt = 0
table_contain_cnt = 0

for i in range(len(dataset)):
    db_path = DATABASE_PATH_PATTERN.format(db_id=dataset[i]["db_id"])

    parsed_query = sqlparse.parse(dataset[i]["query"])[0]
    formatted_query = sqlparse.format(
        str(parsed_query),
        reindent=False,
        keyword_case="upper",
        identifier_case="lower",
        strip_whitespace=True,
    )
    parse = Parser(formatted_query)
    correct_tables = parse.tables
    correct_columns = parse.columns
    print(formatted_query)
    print(correct_columns)
    print(correct_tables)
    print()
    continue
    pred = prompt.get_relevant_tables(db_path, pred_output[i],False, False)
    gold_output = prompt.get_relevant_tables(db_path, dataset[i]["query"], False, False)
    # print(pred)
    # print(gold_output)

    is_equal = equal(pred, gold_output)
    is_table_contain = table_contain(pred, gold_output)
    is_contain = contain(pred, gold_output)
    if is_equal:
        equal_cnt += 1
    if is_table_contain:
        table_contain_cnt += 1
    else:
        print("question: ", dataset[i]["question"])
        print("sql: ", dataset[i]["query"])
        print("pred: ", pred)
        print("gold: ", gold_output)
        print()

    if is_contain:
        contain_cnt += 1


print(equal_cnt)
print(contain_cnt)
print(table_contain_cnt)

SELECT count(*) FROM singer
[]
['singer']

SELECT count(*) FROM singer
[]
['singer']

SELECT name, country, age FROM singer ORDER BY age DESC
['name', 'country', 'age']
['singer']

SELECT name, country, age FROM singer ORDER BY age DESC
['name', 'country', 'age']
['singer']

SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France'
['age', 'country']
['singer']

SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France'
['age', 'country']
['singer']

SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1
['song_name', 'song_release_year', 'age']
['singer']

SELECT song_name, song_release_year FROM singer ORDER BY age LIMIT 1
['song_name', 'song_release_year', 'age']
['singer']

SELECT DISTINCT country FROM singer WHERE age > 20
['country', 'age']
['singer']

SELECT DISTINCT country FROM singer WHERE age > 20
['country', 'age']
['singer']

SELECT country, count(*) FROM singer GROUP BY country
['country']
['singer']

SELECT country, count(*) FRO

In [6]:
with open("./cot_dataset/train/step_1.json","r") as input_file:
    step_1_dataset = json.load(input_file)

for i in range(len(step_1_dataset)):
    size = len(step_1_dataset[i]["instruction"])
    step_1_dataset[i]["instruction"] = step_1_dataset[i]["instruction"][:size-17]
    step_1_dataset[i]["output"] = "[Relevant Tables]\n"+step_1_dataset[i]["output"]
    print(step_1_dataset[i])
    break
with open("./cot_dataset/train/step_1.json","w") as output_file:
    json.dump(step_1_dataset,output_file)

{'instruction': "Given [Database Schema] and [Foreign Keys], your task is to identify the [Relevant Tables] to answer the [Question].\n\n[Database Schema] Every table consists of several columns. Each line describes the name, type of the column and optional value examples. In some cases, column name can be ambiguous, and extra comment is provided to assist in understanding.\n- Table: department. Primary Key: (department_id)\ndepartment_id, INT\nname, TEXT, Value Examples: ['Defense', 'Veterans Affairs', 'Homeland Security']\ncreation, TEXT, Value Examples: ['1965', '1966', '1953']\nranking, INT\nbudget_in_billions, REAL\nnum_employees, REAL\n- Table: head. Primary Key: (head_id)\nhead_id, INT\nname, TEXT, Value Examples: ['Dudley Hart', 'Nick Faldo', 'Tiger Woods']\nborn_state, TEXT, Value Examples: ['Delaware', 'Florida', 'California']\nage, REAL\n- Table: management. Primary Key: (department_id)\ndepartment_id, INT\nhead_id, INT\ntemporary_acting, TEXT, Value Examples: ['No', 'Yes']\

In [1]:
from sql_metadata import Parser
import json
with open("./041601_galore_dev.json", "r") as input_file:
    pred_dataset = json.load(input_file)

fo = open("041601_galore_dev.txt", "w")
for case in pred_dataset:
    position = case.find("SELECT ")
    sql = case[position:].strip()
    # sql = " ".join(sql.split())
    if sql == "":
        sql = "SELECT 1;"
    fo.write(sql + "\n")
fo.close()

In [4]:
import sqlite3
import prompt
import util
from sql_metadata import Parser

with open("./0321_step_2.json", "r") as input_file:
    pred_dataset = json.load(input_file)
with open(DEV_JSON_PATH, "r") as input_file:
    dev_set = json.load(input_file)
with open("spider_dev_comment.json", "r") as input_file:
    comment_map =json.load(input_file)
step_3_set=[]
for i in range(len(pred_dataset)):
    formatted_sql = prompt.format_query(pred_dataset[i])
    parser = Parser(formatted_sql)
    relevant_tables = parser.tables
    relevant_columns = parser.columns
    relevant_tables = [s.lower() for s in relevant_tables]
    relevant_columns = set([s.lower() for s in relevant_columns])
    print(relevant_columns)
    if (len(relevant_tables) == 1) and (formatted_sql.find("YEAR") != -1):
        relevant_columns.add(f"{relevant_tables[0]}.year")
    db_id =dev_set[i]["db_id"]
    db_path = DATABASE_PATH_PATTERN.format(db_id=db_id)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    table_map = {}
    for table_name in relevant_tables:
        column_info = []
        cursor.execute(f"PRAGMA table_info(`{table_name}`);")
        columns = cursor.fetchall()
        primary_keys = []
        for column in columns:
            column_name = column[1].lower()

            if (column_name in relevant_columns) or (
                f"{table_name}.{column_name}" in relevant_columns
            ):
                if column[5] == 1:
                    primary_keys.append(column_name)
                # name,type
                column_info.append((column_name, column[2].upper()))

        table_map[table_name] = (primary_keys, column_info)
    schema_str = prompt.build_schema_str(db_path, table_map,dev_set[i]["question"],comment_map[db_id])
    
    foreign_key = prompt.build_foreign_key_str(db_path, relevant_tables)
    
    instruction = prompt.STEP_2_INSTRUCTION_PATTERN.format(
        schema=schema_str, foreign_key=foreign_key, question=dev_set[i]["question"]
    )
    cursor.close()
    conn.close()
    step_3_set.append(instruction)
with open("0321_step_3.json", "w") as output_file:
    json.dump(step_3_set,output_file)

set()
set()
{'age', 'name', 'country'}
{'age', 'name', 'country'}
{'age', 'country'}
{'age', 'country'}
{'age', 'name', 'song_release_year'}
{'age', 'song_name', 'song_release_year'}
{'age', 'country'}
{'age', 'country'}
{'country'}
{'country'}
{'age', 'song_name'}
{'age', 'song_name'}
{'location', 'name', 'capacity'}
{'location', 'name', 'capacity'}
{'capacity'}
{'capacity'}
{'name', 'average', 'capacity'}
{'name', 'average', 'capacity'}
set()
set()
{'concert.stadium_id', 'stadium.stadium_id', 'stadium.name'}
{'concert.stadium_id', 'stadium.stadium_id', 'stadium.name'}
{'stadium.stadium_id', 'stadium.name', 'concert.stadium_id', 'concert.year', 'stadium.capacity'}
{'stadium.stadium_id', 'stadium.name', 'concert.stadium_id', 'concert.year', 'stadium.capacity'}
set()
set()
{'name', 'stadium_id'}
{'name', 'stadium_id'}
{'age', 'country'}
{'name', 'stadium_id'}
{'name', 'stadium_id'}
{'concert.concert_id', 'singer_in_concert.concert_id', 'concert.theme', 'concert.concert_name'}
{'concert.

In [5]:
import sqlite3


def execute(db_path: str, sql: str):

    conn = sqlite3.connect(db_path)

    # 创建一个游标对象
    cursor = conn.cursor()
    try:
        # 执行一个SQL语句
        cursor.execute(sql)

        # 获取查询结果
        rows = cursor.fetchall()

        return ""
    except sqlite3.Error as e:
        return str(e)

    finally:
        # 关闭游标和数据库连接
        if cursor:
            cursor.close()
        if conn:
            conn.close()


with open("./0315_step_2_final.json", "r") as input_file:
    pred_dataset = json.load(input_file)
with open(DEV_JSON_PATH, "r") as input_file:
    dev = json.load(input_file)
for i in range(len(pred_dataset)):
    db_id = dev[i]["db_id"]
    db_path = DATABASE_PATH_PATTERN.format(db_id=db_id)
    case = pred_dataset[i]
    position = case.find("[SQL Query]\nSELECT")
    sql = case[position + 12 :].strip()
    error = execute(db_path, sql)
    print(sql)
    if error != "":
        print(error)
    # break

SELECT count(*) FROM singer;
SELECT count(*) FROM singer;
SELECT name, country, age FROM singer ORDER BY age DESC;
SELECT name, country, age FROM singer ORDER BY age DESC;
SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France';
SELECT avg(age), min(age), max(age) FROM singer WHERE country = 'France';
SELECT name, song_release_year FROM singer ORDER BY age LIMIT 1;
SELECT name, song_release_year FROM singer WHERE age = (SELECT min(age) FROM singer);
SELECT DISTINCT country FROM singer WHERE age > 20;
SELECT DISTINCT country FROM singer WHERE age > 20;
SELECT country, count(*) FROM singer GROUP BY country;
SELECT country, count(*) FROM singer GROUP BY country;
SELECT song_name FROM singer WHERE age > (SELECT avg(age) FROM singer);
SELECT song_name FROM singer WHERE age > (SELECT avg(age) FROM singer);
SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000;
SELECT LOCATION, name FROM stadium WHERE capacity BETWEEN 5000 AND 10000;
SELECT max(capacity), a

In [3]:
import json
with open("./0325_step1_test.json", "r") as input_file:
    pred_dataset = json.load(input_file)
with open("cot_dataset/test/step_1.json", "r") as input_file:
    dev = json.load(input_file)


def get_tables(out: str):
    position = out.find("[Relevant Tables]\n")
    if  position!=-1:
        sql = out[position + 18 :].strip()  
    else:
        sql = out
    sql=sql.replace("[","").replace("]","").lower()
            # print(sql)
    return set(sql.split(","))


error_count = 0
uncontain = 0
tables = []
for i in range(len(pred_dataset)):

    
    gold_output = dev[i]["output"]
    pred_output = pred_dataset[i]
    gold_tables = get_tables(gold_output)
    pred_tables = get_tables(pred_output)
    tables.append(list(pred_tables))
    if gold_tables != pred_tables:
        error_count += 1

        # print(gold_tables)
        # print(pred_tables)
    if not (gold_tables.issubset(pred_tables)):
        uncontain+=1
        print(i)
        print(gold_tables)
        print(pred_tables)
        print("raw",pred_output)
        print("----")
print(1-(error_count)/len(pred_dataset))
print(1-uncontain/len(pred_dataset))
with open("0328_step_1_output.json","w") as output:
    json.dump(tables, output)

77
{'orders', 'products', 'order_items'}
{'orders', 'shipments', 'invoices'}
raw [Relevant Tables]
orders,shipments,invoices
----
181
{'vehicle', 'vehicle_driver', 'driver'}
{'vehicle_driver', 'vehicle'}
raw [Relevant Tables]
vehicle_driver,vehicle
----
182
{'vehicle', 'vehicle_driver', 'driver'}
{'vehicle_driver', 'vehicle'}
raw [Relevant Tables]
vehicle_driver,vehicle
----
374
{'customers_and_services'}
{'services', 'customers'}
raw [Relevant Tables]
customers,services
----
378
{'services', 'customers', 'customer_interactions', 'customers_and_services'}
{'services', 'customer_interactions', 'customers_and_services'}
raw [Relevant Tables]
services,customers_and_services,customer_interactions
----
380
{'services', 'customers', 'customer_interactions', 'customers_and_services'}
{'customer_interactions', 'customers', 'services'}
raw [Relevant Tables]
customer_interactions,customers,services
----
449
{'teachers', 'classes'}
{'classes'}
raw [Relevant Tables]
classes
----
450
{'teachers', '